   # Introduction
   
   ### In this notebook, we're going to obtain some useful information about all stocks that are negotiated in brazilian stock exchange (B3). This data can be useful for some big picture analysis in Python environment. We have a few steps for this to be possible:
   
   1) We're gonna have to obtain a list of all stock names, codes and their respective economic sectors. This list is available in B3 official website (http://www.b3.com.br/pt_br/produtos-e-servicos/negociacao/renda-variavel/consultas.htm), and I've already obtained it and made it more "Pandas friendly" with some simple handwork;
   
   2) There's a first problem that we will face: in this file, the codes aren't followed by their respective numbers (the stock name is composed by a code followed by a number). We're gonna have do some web scraping in Fundamentus website to obtain all of these different codes;
   
   3) Having all the numbers, we will finally construct a huge dataframe also via scraping, with all negotiated stocks, their respective economic sectors and indicators for advanced fundamental analysis.
    

## First part

For this first part, we're only going to import the needed libraries and also import the csv file containing the stock codes (without numbers) and the respective economic sectors, transforming it in a Pandas DataFrame

In [1]:
# Libraries needed:

import numpy as np
import pandas as pd
import requests
from scrapy import Selector
from time import sleep

# setting the maximum number of columns to display:
pd.set_option('display.max_columns', 60)

In [2]:
# importing the first csv file, containing all codes without the numbers (and dropping the first column)

df1 = pd.read_csv('companies_and_sectors.csv').drop(['Unnamed: 0'], axis=1)


# this is how the initial dataframe looks like:

df1.head()

,Nome,Código,Setor econômico,Subsetor,Segmento
0,3R PETROLEUM,RRRP,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,COSAN,CSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
2,DOMMO,DMMO,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
3,ENAUTA PART,ENAT,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
4,PET MANGUINH,RPMG,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"


In [3]:
# we can check how many stocks we're talking about:

df1.shape

(429, 5)

In [4]:
# lastly, we can have some idea of the categorical values:

df1['Setor econômico'].value_counts()

Consumo Cíclico                    88
Financeiro                         81
Bens Industriais                   75
Utilidade Pública                  65
Materiais Básicos                  30
Consumo não Cíclico                25
Saúde                              21
Outros                             17
Petróleo, Gás e Biocombustíveis    11
Tecnologia da Informação           10
Comunicações                        6
Name: Setor econômico, dtype: int64

## Second part

For this second part, we're going to scrape the website http://www.fundamentus.com.br/index.php. The following code will first make a simple search based on the given code (string) and return a list with all stock codes (with numbers) that contains that code. For example, if we pass the code "PETR", the function will return 2 stock codes: PETR3 and PETR4. For that, we need a user agent that is different from the default python requests user agent. If you don't know yours, just Google "my user agent" and copy the weird code that will pop up. Then, put this on the variable "my_UA" as a string. The requests.get() will pick up the HTML code of the website, and the Selector object will crawl through this HTML code based on the given "xpath" that we give it and return what we wanted. The xpath is basically an specific "adress" for each specific information on the website. For more information about xpath syntax, check out: https://docs.scrapy.org/en/latest/topics/selectors.html

In [5]:
my_UA = '' # enter here your user agent!

#there is two parameters for this function:

def obtain_codes(string, user_agent): #the code (without number) of the stock and the user agent used in requests
    
    url = 'http://fundamentus.com.br/detalhes.php?papel=' + string # this will make a search for the specific code
    
    xpath = '//tbody//td//a//text()'
        
    headers = {'User-Agent': user_agent}
    
    html_code = requests.get(url, headers=headers).content
    
    selectorObject = Selector(text=html_code)
    
    all_codes_list = selectorObject.xpath(xpath).extract()
    
    codes_list = [code for code in all_codes_list if string in code]
    
    # we need to filter the "all_codes_list" and pick up only those codes containing our first string
    
    return codes_list
    

In [6]:
# you can try this out with whatever code on the dataframe

print('VALE stock names: {}'.format(obtain_codes('VALE', my_UA)))

# some codes will not return any stock at all. It means that either Fundamentus website
# doesn't have any  information or that those stocks aren't regularly negotiated

print('CMSA stock names: {}'.format(obtain_codes('CMSA', my_UA)))

VALE stock names: ['VALE3', 'VALE5']
CMSA stock names: []


#### We'll then construct a new dataframe, now with the stock numbers, using the "obtain_codes" function:

For that, we'll iterate through the codes column of the original dataframe and, for each code, we're going to search for all stocks with that code in their names. Then, we'll construct the rows of the new dataframe with each new stock. As we'll put a "sleep" of .5 second for every iteration, it will last .5 * 428 seconds = around 4 minutes to build our DataFrame.

In [7]:
# empty new DataFrame:
df2 = pd.DataFrame(columns=df1.columns)

stocks_count = 0

# iterating in "codes" column of the DataFrame:

for index, code in enumerate(df1['Código']): # iterating by codes and indexes, from 0 to 428
    
    # we'll first get all the information for each specific code and save on a dictionary
    values_dict = dict(df1[['Nome', 'Setor econômico', 'Subsetor', 'Segmento']].loc[index])
    
    list_of_codes = obtain_codes(code, my_UA) # obtaining list of codes
    
    # making the exception so it won't get any empty list
    
    if len(list_of_codes) != 0:
        
        for numcode in list_of_codes: # generating a new dict for each numerical code
            
            values_dict['Código'] = numcode
            
            df2.loc[stocks_count] = values_dict # constructing the dataframe row with the dict
            
            stocks_count += 1
    
    sleep(0.5) # good practice whenever scraping, prevents DDOSing and other kinds of "bans" from websites

In [8]:
# This is how our second DataFrame looks like, and its shape

print('Shape of the new DataFrame: {}'.format(df2.shape))

df2.head()

Shape of the new DataFrame: (537, 5)


,Nome,Código,Setor econômico,Subsetor,Segmento
0,3R PETROLEUM,RRRP3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
1,COSAN,CSAN3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
2,DOMMO,DMMO3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
3,ENAUTA PART,ENAT3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"
4,PET MANGUINH,RPMG3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição"


## Third part

In the last part, we're going to search specific information about each stock, and finally construct a bigger DataFrame with all this data. For that, we'll also crawl through Fundamentus website, now using a more complex xpath database instead of only one string. In this database, we have the first column as the name of the stock fundamental indicator and the second one as the respective xpath for that indicator.

In [9]:
xpaths_df = pd.read_csv('xpaths.csv').drop(['Unnamed: 0'], axis=1) #dropping again the first column

xpaths_df.head(4)

# this is how our dataframe looks like. It will guide the script to the right data to get on the website.

,indicator,xpath
0,EBIT/ativo,"//table[@class=""w728""][3]//tr[7]/td[6]/span[@c..."
1,EBIT12M,"//table[@class=""w728""][5]/tr[4]/td[2]//span[@c..."
2,EBIT3M,"//table[@class=""w728""][5]/tr[4]/td[4]//span[@c..."
3,EV/EBIT,"//table[@class=""w728""][3]//tr[11]/td[4]/span[@..."


In [10]:
# The supported indicators are as follows:

xpaths_df['indicator'].unique()

# you can check Fundamentus website for more information about each indicator


# to check the index for each indicator, run the following code:
# dict(enumerate(xpaths_df['indicator']))

array(['EBIT/ativo', 'EBIT12M', 'EBIT3M', 'EV/EBIT', 'EV/EBITDA ', 'LPA',
       'P/Ativos', 'P/EBIT', 'P/L', 'P/VP', 'P/ativcircLiq', 'P/capGiro',
       'PSR', 'ROE', 'ROIC', 'VPA', 'ativo', 'ativoCirc', 'cresRec5A',
       'disponib', 'divBr/patrim', 'divBruta', 'divLiq', 'divYield',
       'giroAtivos', 'liquidCorr', 'lucroLiq12M', 'lucroLiq3M',
       'margBruta', 'margEBIT', 'margLiq', 'max1ano', 'min1ano', 'numacs',
       'osc12M', 'osc2016', 'osc2017', 'osc2018', 'osc2019', 'osc2020',
       'osc2021', 'osc30D', 'oscDia', 'oscMes', 'patrimLiq', 'preço',
       'recLiq12M', 'recLiq3M', 'valorFirma', 'valorMercado'],
      dtype=object)

In the following cell, we're going to first call an external file containing an useful function, that clean all the indicators values. Doing this, we won't obtain any value containing typos or not convertible to numbers. The function "fundamental_indicators" will obtain all given indicators (by xpath) for one specific stock code and user agent.

In [11]:
# the file that we'll be running below contains a function that "filter" the values
# this way, we won't get any messed up value, containing typos like '\n' or commas

%run clean_values.py
    
# we'll use the following function to obtain the required indicators

# "string" is the stock code, "df" is the dataframe with the required indicators and respective xpaths and 
# "user_agent" is the local user agent

# the function will return a dictionary with the indicator names and values

def fundamental_indicators(string, df, user_agent):
    
    values_dict = dict()
    
    url = 'http://fundamentus.com.br/detalhes.php?papel=' + string # this will make a search for the specific stock
    
    headers = {'User-Agent': user_agent}
    
    html_code = requests.get(url, headers=headers).content
    
    selectorObject = Selector(text=html_code)
    
    for indicator, xpath in zip(df.iloc[:,0], df.iloc[:,1]): # iterating in each column of the DataFrame
        
        try:
            
            value = selectorObject.xpath(xpath).extract()[0] # extracting the value
        
        except:
            
            value = np.nan
        
        values_dict[indicator] = clean_values(value) # cleaning the value with our function
    
    return values_dict

In [12]:
# testing the function for given specific indicators (price swings in last years, month and day)

fundamental_indicators('ABEV3', xpaths_df.iloc[34:44,:], my_UA)

{'osc12M': -0.124,
 'osc2016': -0.015,
 'osc2017': 0.331,
 'osc2018': -0.256,
 'osc2019': 0.241,
 'osc2020': -0.142,
 'osc2021': 0.029,
 'osc30D': 0.016,
 'oscDia': -0.015,
 'oscMes': 0.029}

Finally, constructing the final DataFrame with all possible data! This process should take around 10 minutes.

In [13]:
# constructing raw dataframe first:
indicators_df = pd.DataFrame(columns=xpaths_df['indicator'])

for index, stock in enumerate(df2['Código']):

    values_dict = fundamental_indicators(stock, xpaths_df, my_UA) # dictionary with all indicators for the stock
    
    indicators_df.loc[index] = values_dict # assigning the dictionary as a DataFrame row
    
    sleep(0.5)

In [14]:
# The constructed DataFrame of indicators looks like this:

indicators_df.head(3)

indicator,EBIT/ativo,EBIT12M,EBIT3M,EV/EBIT,EV/EBITDA,LPA,P/Ativos,P/EBIT,P/L,P/VP,P/ativcircLiq,P/capGiro,PSR,ROE,ROIC,VPA,ativo,ativoCirc,cresRec5A,disponib,divBr/patrim,divBruta,divLiq,divYield,giroAtivos,liquidCorr,lucroLiq12M,lucroLiq3M,margBruta,margEBIT,margLiq,max1ano,min1ano,numacs,osc12M,osc2016,osc2017,osc2018,osc2019,osc2020,osc2021,osc30D,oscDia,oscMes,patrimLiq,preço,recLiq12M,recLiq3M,valorFirma,valorMercado
0,0.000,0.000000e+00,-272.0,NaN,NaN,0.00,9.70,NaN,0.00,17.80,-25.08,160.68,NaN,NaN,NaN,1.99,4.008570e+08,2.747300e+07,NaN,1.833600e+07,NaN,0.000000e+00,-1.833600e+07,0.000,NaN,8.38,0.000000e+00,-12749000.0,NaN,NaN,NaN,37.67,20.69,109504000.0,0.708,0.000,0.000,0.000,0.000,0.789,-0.046,0.048,-0.003,-0.046,2.183640e+08,35.50,0.000000e+00,1.030400e+07,3.869070e+09,3.887400e+09
1,0.053,1.902040e+09,618022000.0,22.90,17.40,2.60,0.88,16.74,31.09,2.99,-2.44,5.58,2.45,0.096,0.064,27.04,3.610080e+10,1.174320e+10,0.169,4.845150e+09,1.55,1.656240e+10,1.171730e+10,0.019,0.36,1.95,1.024250e+09,303839000.0,0.296,0.146,0.081,90.40,45.04,394210000.0,0.037,0.705,0.116,-0.171,1.125,0.112,0.067,0.061,-0.019,0.067,1.065970e+10,80.77,1.301430e+10,3.714920e+09,4.355760e+10,3.184030e+10
2,-0.350,-1.231010e+08,-27547000.0,-2.28,-4.45,-1.34,0.96,-2.75,-0.82,-0.39,-0.33,-1.24,0.91,0.475,-1.131,-2.82,3.521000e+08,1.916550e+08,0.115,6.373200e+07,-0.01,5.235000e+06,-5.849700e+07,0.000,1.05,0.41,-4.121660e+08,185553000.0,-0.140,-0.332,-1.112,2.29,0.47,307949000.0,-0.523,-0.669,-0.531,-0.058,-0.701,-0.519,-0.209,-0.236,-0.026,-0.209,-8.675690e+08,1.10,3.706450e+08,9.958700e+07,2.802470e+08,3.387440e+08


In [15]:
# Finally, joining the original dataframe with our new one:

final_df = df2.join(indicators_df)

In [16]:
# This is how our so desired dataframe looks like:

final_df.head()

,Nome,Código,Setor econômico,Subsetor,Segmento,EBIT/ativo,EBIT12M,EBIT3M,EV/EBIT,EV/EBITDA,LPA,P/Ativos,P/EBIT,P/L,P/VP,P/ativcircLiq,P/capGiro,PSR,ROE,ROIC,VPA,ativo,ativoCirc,cresRec5A,disponib,divBr/patrim,divBruta,divLiq,divYield,giroAtivos,liquidCorr,lucroLiq12M,lucroLiq3M,margBruta,margEBIT,margLiq,max1ano,min1ano,numacs,osc12M,osc2016,osc2017,osc2018,osc2019,osc2020,osc2021,osc30D,oscDia,oscMes,patrimLiq,preço,recLiq12M,recLiq3M,valorFirma,valorMercado
0,3R PETROLEUM,RRRP3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",0.000,0.000000e+00,-272.0,NaN,NaN,0.00,9.70,NaN,0.00,17.80,-25.08,160.68,NaN,NaN,NaN,1.99,4.008570e+08,2.747300e+07,NaN,1.833600e+07,NaN,0.000000e+00,-1.833600e+07,0.000,NaN,8.38,0.000000e+00,-12749000.0,NaN,NaN,NaN,37.67,20.69,109504000.0,0.708,0.000,0.000,0.000,0.000,0.789,-0.046,0.048,-0.003,-0.046,2.183640e+08,35.50,0.000000e+00,1.030400e+07,3.869070e+09,3.887400e+09
1,COSAN,CSAN3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",0.053,1.902040e+09,618022000.0,22.90,17.40,2.60,0.88,16.74,31.09,2.99,-2.44,5.58,2.45,0.096,0.064,27.04,3.610080e+10,1.174320e+10,0.169,4.845150e+09,1.55,1.656240e+10,1.171730e+10,0.019,0.36,1.95,1.024250e+09,303839000.0,0.296,0.146,0.081,90.40,45.04,394210000.0,0.037,0.705,0.116,-0.171,1.125,0.112,0.067,0.061,-0.019,0.067,1.065970e+10,80.77,1.301430e+10,3.714920e+09,4.355760e+10,3.184030e+10
2,DOMMO,DMMO3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",-0.350,-1.231010e+08,-27547000.0,-2.28,-4.45,-1.34,0.96,-2.75,-0.82,-0.39,-0.33,-1.24,0.91,0.475,-1.131,-2.82,3.521000e+08,1.916550e+08,0.115,6.373200e+07,-0.01,5.235000e+06,-5.849700e+07,0.000,1.05,0.41,-4.121660e+08,185553000.0,-0.140,-0.332,-1.112,2.29,0.47,307949000.0,-0.523,-0.669,-0.531,-0.058,-0.701,-0.519,-0.209,-0.236,-0.026,-0.209,-8.675690e+08,1.10,3.706450e+08,9.958700e+07,2.802470e+08,3.387440e+08
3,ENAUTA PART,ENAT3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",0.070,3.211600e+08,73717000.0,5.03,1.92,1.30,0.68,9.76,9.05,1.07,8.70,2.14,2.69,0.118,0.119,11.04,4.579040e+09,2.003430e+09,0.277,1.742280e+09,0.08,2.243810e+08,-1.517900e+09,0.097,0.25,3.72,3.461400e+08,39407000.0,0.334,0.276,0.298,16.33,5.67,265807000.0,-0.276,-0.010,1.070,-0.002,0.947,-0.159,-0.005,0.058,-0.028,-0.005,2.935820e+09,11.79,1.162900e+09,2.244610e+08,1.615960e+09,3.133860e+09
4,PET MANGUINH,RPMG3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",0.000,0.000000e+00,-76542000.0,NaN,NaN,0.00,0.07,NaN,0.00,-0.06,-0.06,-0.07,NaN,NaN,NaN,-45.46,2.990940e+09,2.761940e+09,0.121,1.900000e+01,NaN,0.000000e+00,-1.900000e+01,0.000,NaN,0.50,0.000000e+00,-106412000.0,NaN,NaN,NaN,5.53,2.05,67692000.0,-0.462,0.897,0.096,-0.621,0.712,-0.449,0.039,0.014,0.007,0.039,-3.076960e+09,2.93,0.000000e+00,4.323390e+08,1.983190e+08,1.983380e+08


In [17]:
# finally exporting it to a file

final_df.to_csv('stock_data.csv')